In [110]:
using System;
using System.Threading;

class DefiniteIntegral
{
    private static object lockObject = new object();

    public static double Solve(double a, double b, Func<double, double> function, double step, int threadsnumber)
    {
        double result = 0.0;
        double intervalWidth = (b - a) / threadsnumber;
        ManualResetEvent[] doneThread = new ManualResetEvent[threadsnumber];

        for (int i = 0; i < threadsnumber; i++)
        {
            doneThread[i] = new ManualResetEvent(false);
            int threadIndex = i;
            ThreadPool.QueueUserWorkItem((state) =>
            {
                double startPoint = a + threadIndex * intervalWidth;
                double endPoint = startPoint + intervalWidth;
                double sum = (function(startPoint) + function(endPoint)) / 2.0;

                for (double x = startPoint + step / 2; x < endPoint; x += step / 2)
                {
                    sum += function(x);
                }

                sum *= step / 2;

                lock (lockObject)
                {
                    result += sum;
                }

                doneThread[threadIndex].Set();
            });
        }

        WaitHandle.WaitAll(doneThread);

        return result;
    }
}

In [111]:
#r "nuget: xunit, 2.8.1"

using Xunit;

var X = (double x) => x;
var SIN = (double x) => Math.Sin(x);

Assert.Equal(0, DefiniteIntegral.Solve(-1, 1, X, 1e-4, 2), 1e-4);

Assert.Equal(0, DefiniteIntegral.Solve(-1, 1, SIN, 1e-5, 8), 1e-4);

Assert.Equal(50, DefiniteIntegral.Solve(0, 10, X, 1e-6, 8), 1e-5);

Installed Packages xunit, 2.8.1